In [ ]:
import os
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Adding AWS S3 Minio configs
sparkConf = (
    SparkConf()
    .set("spark.jars.ivy","/home/brijeshdhaker/.ivy2")
    #.set("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.0.0")
    #.set("spark.executor.heartbeatInterval", "300000")
    #.set("spark.network.timeout", "400000")
    #.set("spark.hadoop.fs.s3a.endpoint", "http://minio.sandbox.net:9010")
    #.set("spark.hadoop.fs.s3a.access.key", "pgm2H2bR7a5kMc5XCYdO")
    #.set("spark.hadoop.fs.s3a.secret.key", "zjd8T0hXFGtfemVQ6AH3yBAPASJNXNbVSx5iddqG")
    #.set("spark.hadoop.fs.s3a.path.style.access", "true")
    #.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    #.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    #.set("spark.sql.warehouse.dir", "s3a://defaultfs/spark/warehouse")
    #.set("spark.hadoop.fs.defaultFS", "s3a://defaultfs/")
    #.set("spark.eventLog.enabled", "true")
    #.set("spark.eventLog.dir", "file:///apps/var/logs/spark-events")
)

spark = (
    SparkSession.builder.master("local[*]").
        appName('spark-hints').
        config(conf=sparkConf).
        getOrCreate()
)

spark.sparkContext.setLogLevel('ERROR')
spark

25/10/13 08:50:38 WARN Utils: Your hostname, vmware-ubuntu-24.04 resolves to a loopback address: 127.0.1.1; using 192.168.154.133 instead (on interface ens33)
25/10/13 08:50:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/13 08:50:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
employee_columns = ['emp_id', 'emp_name', 'emp_role', 'emp_manager', 'emp_hiredate', 'emp_salary', 'emp_comm', 'emp_dept']

employee_schema = StructType() \
    .add("emp_id", IntegerType(), True) \
    .add("emp_name", StringType(), True) \
    .add("emp_role", StringType(), True) \
    .add("emp_manager", StringType(), True) \
    .add("emp_hiredate", DateType(), True) \
    .add("emp_salary", IntegerType(), True) \
    .add("emp_comm", IntegerType(), True) \
    .add("emp_dept", IntegerType(), True)

employee_df = spark.read.csv("file:///apps/sandbox/defaultfs/employee.csv",
    header=True,
    schema=employee_schema
)

#
# employee_df.printSchema()

#
# print(employee_df.rdd.getNumPartitions())

#
employee_df.show(truncate=False)

+------+--------+---------+-----------+------------+----------+--------+--------+
|emp_id|emp_name|emp_role |emp_manager|emp_hiredate|emp_salary|emp_comm|emp_dept|
+------+--------+---------+-----------+------------+----------+--------+--------+
|7839  |KING    |PRESIDENT|NULL       |1981-11-17  |5000      |NULL    |10      |
|7698  |BLAKE   |MANAGER  |7839       |1981-01-05  |2850      |125     |30      |
|7782  |CLARK   |MANAGER  |7839       |1983-09-06  |2450      |NULL    |10      |
|7566  |JONES   |MANAGER  |7839       |1981-02-04  |2975      |345     |20      |
|7788  |SCOTT   |ANALYST  |7566       |1984-12-17  |3000      |NULL    |20      |
|7902  |FORD    |ANALYST  |7566       |1981-11-27  |3000      |NULL    |40      |
|7369  |SMITH   |CLERK    |7902       |1980-04-04  |800       |NULL    |20      |
|7499  |ALLEN   |SALESMAN |7698       |1981-05-05  |1600      |300     |30      |
|7521  |WARD    |SALESMAN |7698       |1982-06-23  |1250      |500     |50      |
|7654  |MARTIN  

In [3]:
dept_columns = ['dept_id', 'dept_name', 'dept_location']

dept_schema = StructType() \
    .add("dept_id", IntegerType(), True) \
    .add("dept_name", StringType(), True) \
    .add("dept_location", StringType(), True)

dept_df = spark.read.format("csv") \
    .option("header", True) \
    .schema(dept_schema) \
    .load("file:///apps/sandbox/defaultfs/departments.csv")

#dept_df.printSchema()

dept_df.show(truncate=False)

+-------+----------+-------------+
|dept_id|dept_name |dept_location|
+-------+----------+-------------+
|10     |ACCOUNTING|NY           |
|20     |RESEARCH  |TX           |
|30     |SALES     |CH           |
|40     |OPERATIONS|TX           |
|50     |ADMIN     |CA           |
|70     |SECURITY  |FL           |
+-------+----------+-------------+



#### 1. Join Strategy Hints: 
These hints influence how Spark performs joins between DataFrames.

In [ ]:
#
# BROADCAST: Suggests that Spark broadcast a smaller DataFrame to all worker nodes when performing a join, which can be beneficial for small DataFrames to avoid shuffling.
#
employee_df.join(dept_df.hint("broadcast"), employee_df["emp_dept"] == dept_df["dept_id"]).show(truncate=False)

+------+--------+---------+-----------+------------+----------+--------+--------+-------+----------+-------------+
|emp_id|emp_name| emp_role|emp_manager|emp_hiredate|emp_salary|emp_comm|emp_dept|dept_id| dept_name|dept_location|
+------+--------+---------+-----------+------------+----------+--------+--------+-------+----------+-------------+
|  7839|    KING|PRESIDENT|       NULL|  1981-11-17|      5000|    NULL|      10|     10|ACCOUNTING|           NY|
|  7698|   BLAKE|  MANAGER|       7839|  1981-01-05|      2850|     125|      30|     30|     SALES|           CH|
|  7782|   CLARK|  MANAGER|       7839|  1983-09-06|      2450|    NULL|      10|     10|ACCOUNTING|           NY|
|  7566|   JONES|  MANAGER|       7839|  1981-02-04|      2975|     345|      20|     20|  RESEARCH|           TX|
|  7788|   SCOTT|  ANALYST|       7566|  1984-12-17|      3000|    NULL|      20|     20|  RESEARCH|           TX|
|  7902|    FORD|  ANALYST|       7566|  1981-11-27|      3000|    NULL|      40

In [ ]:
#
# MERGE: Suggests a sort-merge join strategy, which is often efficient for large DataFrames that are already sorted or can be efficiently sorted.
#
employee_df.join(dept_df.hint("merge"), employee_df["emp_dept"] == dept_df["dept_id"]).show(truncate=False)

+------+--------+---------+-----------+------------+----------+--------+--------+-------+----------+-------------+
|emp_id|emp_name| emp_role|emp_manager|emp_hiredate|emp_salary|emp_comm|emp_dept|dept_id| dept_name|dept_location|
+------+--------+---------+-----------+------------+----------+--------+--------+-------+----------+-------------+
|  7839|    KING|PRESIDENT|       NULL|  1981-11-17|      5000|    NULL|      10|     10|ACCOUNTING|           NY|
|  7782|   CLARK|  MANAGER|       7839|  1983-09-06|      2450|    NULL|      10|     10|ACCOUNTING|           NY|
|  7935|  ROBERT|  ANALYST|       7566|  1984-03-31|      1940|     225|      10|     10|ACCOUNTING|           NY|
|  7566|   JONES|  MANAGER|       7839|  1981-02-04|      2975|     345|      20|     20|  RESEARCH|           TX|
|  7788|   SCOTT|  ANALYST|       7566|  1984-12-17|      3000|    NULL|      20|     20|  RESEARCH|           TX|
|  7369|   SMITH|    CLERK|       7902|  1980-04-04|       800|    NULL|      20

In [8]:
#
# SHUFFLE\_MERGE: Similar to MERGE but explicitly suggests shuffling both DataFrames before performing a sort-merge join.
#

employee_df.join(dept_df.hint("shuffle_merge"), employee_df["emp_dept"] == dept_df["dept_id"]).show(truncate=False)

+------+--------+---------+-----------+------------+----------+--------+--------+-------+----------+-------------+
|emp_id|emp_name|emp_role |emp_manager|emp_hiredate|emp_salary|emp_comm|emp_dept|dept_id|dept_name |dept_location|
+------+--------+---------+-----------+------------+----------+--------+--------+-------+----------+-------------+
|7839  |KING    |PRESIDENT|NULL       |1981-11-17  |5000      |NULL    |10      |10     |ACCOUNTING|NY           |
|7782  |CLARK   |MANAGER  |7839       |1983-09-06  |2450      |NULL    |10      |10     |ACCOUNTING|NY           |
|7935  |ROBERT  |ANALYST  |7566       |1984-03-31  |1940      |225     |10      |10     |ACCOUNTING|NY           |
|7566  |JONES   |MANAGER  |7839       |1981-02-04  |2975      |345     |20      |20     |RESEARCH  |TX           |
|7788  |SCOTT   |ANALYST  |7566       |1984-12-17  |3000      |NULL    |20      |20     |RESEARCH  |TX           |
|7369  |SMITH   |CLERK    |7902       |1980-04-04  |800       |NULL    |20      

In [9]:
#
# SHUFFLE\_HASH: Suggests a shuffle hash join, where both DataFrames are shuffled and then joined using a hash table on each partition.
#
employee_df.join(dept_df.hint("shuffle_hash"), employee_df["emp_dept"] == dept_df["dept_id"]).show(truncate=False)

+------+--------+---------+-----------+------------+----------+--------+--------+-------+----------+-------------+
|emp_id|emp_name|emp_role |emp_manager|emp_hiredate|emp_salary|emp_comm|emp_dept|dept_id|dept_name |dept_location|
+------+--------+---------+-----------+------------+----------+--------+--------+-------+----------+-------------+
|7566  |JONES   |MANAGER  |7839       |1981-02-04  |2975      |345     |20      |20     |RESEARCH  |TX           |
|7788  |SCOTT   |ANALYST  |7566       |1984-12-17  |3000      |NULL    |20      |20     |RESEARCH  |TX           |
|7902  |FORD    |ANALYST  |7566       |1981-11-27  |3000      |NULL    |40      |40     |OPERATIONS|TX           |
|7369  |SMITH   |CLERK    |7902       |1980-04-04  |800       |NULL    |20      |20     |RESEARCH  |TX           |
|7844  |TURNER  |SALESMAN |7698       |1982-09-08  |1500      |0       |40      |40     |OPERATIONS|TX           |
|7876  |ADAMS   |CLERK    |7788       |1980-11-14  |1100      |100     |20      

#### 2. Partitioning Hints: 
These hints help control the number of partitions and their distribution, impacting output file size and parallelism.

In [ ]:
#
# COALESCE: Reduces the number of partitions to a specified number.
#
employee_df.hint("coalesce", 10).write.parquet("output_path")

In [ ]:
#
# REPARTITION: Repartitions the DataFrame to a specified number of partitions, optionally by specific columns.
#
employee_df.hint("repartition", 10, "column_name").write.parquet("output_path")

In [ ]:
#
# REPARTITION\_BY\_RANGE: Repartitions the DataFrame by range based on specified columns, ensuring data within a range is in the same partition.
#
employee_df.hint("repartition_by_range", "column_name").write.parquet("output_path")

In [ ]:
#
# REBALANCE: Rebalances the partitions to achieve a more even distribution of data size across partitions.
#
employee_df.hint("rebalance").write.parquet("output_path")